In [47]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [48]:
start_case = 500
case_num = 500

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    min_k = 4
    max_k = 5.5
    k_p = 0.85
    start_step = 0.75
    end_step = 1.8
    step_cnt = 5
    v_prob_mean_w = 1.460
    v_prob_min_var = 0.027
    p_max = 0.9
    p_min = 0.4

    # min_k = trial.suggest_float("max_k", low=1.5, high=4.5)
    # max_k = trial.suggest_float("min_k", low=5.0, high=7.0)
    # k_p = trial.suggest_float("k_p", low=0.5, high=2.0)
    # start_step = trial.suggest_float("start_step", low=0.2, high=1.5)
    # end_step = trial.suggest_float("end_step", low=max(1.2, start_step + 0.1), high=3.2)
    # step_cnt = trial.suggest_int("step_cnt", low=2, high=12)
    # v_prob_mean_w = trial.suggest_float("v_prob_mean_w", low=0.0, high=3.0)
    # v_prob_min_var = trial.suggest_float("v_prob_min_var", low=0.01, high=0.5)
    # p_max = trial.suggest_float("p_max", low=0.5, high=0.99)
    # p_min = trial.suggest_float("p_min", low=0.1, high=min(0.9, p_max + 0.01))


    delta_max_dist = trial.suggest_int("delta_max_dist", low=1, high=3)
    neighbor_p = trial.suggest_float("neighbor_p", low=1., high=3.)
    duplicate_min_d = trial.suggest_int("duplicate_min_d", low=1, high=5)
    step_p = trial.suggest_float("step_p", low=0.5, high=2.)
    out_step_size = trial.suggest_float("out_step_size", low=50., high=200.)
    start_temp_coef = trial.suggest_float("start_temp_coef", low=1e2, high=1e4)
    end_temp_coef = trial.suggest_float("end_temp_coef", low=1e4+1, high=1e6)

    args = " ".join(
        map(
            str,
            [
                min_k,
                max_k,
                k_p,
                start_step,
                end_step,
                step_cnt,
                v_prob_mean_w,
                v_prob_min_var,
                p_max,
                p_min,
                delta_max_dist,
                neighbor_p,
                duplicate_min_d,
                step_p,
                out_step_size,
                start_temp_coef,
                end_temp_coef
            ],
        )
    )

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-anneal-01-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score(
        ignore_solver_prefix="optuna-anneal-01-"
    )
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-anneal-02",
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "min_k": 4.0,
        "max_k": 5.5,
        "k_p": 0.85,
        "start_step": 0.75,
        "end_step": 1.8,
        "step_cnt": 5,
        "v_prob_mean_w": 1.0,
        "v_prob_min_var": 0.1,
        "p_max": 0.9,
        "p_min": 0.5,
        "delta_max_dist": 2,
        "neighbor_p": 2.,
        "duplicate_min_d": 1,
        "step_p": 1.0,
        "out_step_size": 100.0,
        "start_temp_coef": 1e3,
        "end_temp_coef": 1e5,
    }
)
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.01s
[I 2024-02-19 12:33:19,243] Using an existing study with name 'tune-anneal-01' instead of creating a new one.


start optimize


/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/100 [00:00<?, ?it/s]

/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/trial/_trial.py:648: RuntimeWarning:

Inconsistent parameter values for distribution with name "out_step_size"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 50.0, 'high': 200.0, 'log': False}



[W 2024-02-19 12:33:31,651] Trial 1 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/3n/z49mbd414cx0_x991z2fyqjc0000gn/T/ipykernel_19556/1989338410.py", line 79, in objective
    runner.run(cases=cases, verbose=0)
  File "/Users/tatsuyaishii/dev/heuristics-contests/ahc030/notebooks/../run.py", line 64, in run
    results = Parallel(n_jobs=-1, verbose=verbose)(
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/joblib/parallel.py", line 1056, in __call__
    self.retrieve()
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/joblib/parallel.py", line 935, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/job

KeyboardInterrupt: 

In [ ]:
study = optuna.load_study(
    storage="sqlite:///ahc030.db",
    study_name="tune-steps-03",
)
study.best_params

{'end_step': 1.4642511974988384,
 'start_step': 0.7476624349317118,
 'step_cnt': 4}

In [ ]:
optuna.visualization.plot_contour(study, params=["end_step", "start_step"])

In [ ]:
optuna.visualization.plot_optimization_history(study)